# Libraries

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForQuestionAnswering, pipeline
import torch
import re

# Model

## Creating the Q&A pipeline

In [3]:
nlp = pipeline(
    'question-answering',
    model = 'deepset/roberta-base-squad2',
    tokenizer = 'deepset/roberta-base-squad2'
)

nlp

Downloading: 100%|██████████| 571/571 [00:00<00:00, 151kB/s]
Downloading: 100%|██████████| 473M/473M [00:36<00:00, 13.7MB/s] 
Downloading: 100%|██████████| 79.0/79.0 [00:00<00:00, 28.5kB/s]
Downloading: 100%|██████████| 878k/878k [00:00<00:00, 1.74MB/s]
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 1.01MB/s]
Downloading: 100%|██████████| 772/772 [00:00<00:00, 526kB/s]


In [46]:
context = """
Plot
See also: List of The L Word episodes
Season 1
Main article: The L Word (season 1)
The first season of The L Word premiered on January 18, 2004 and ended on April 11, 2004. The season introduces Bette Porter and Tina Kennard, a couple in a seven-year relationship attempting to have a child; Marina Ferrer, owner of the local cafe The Planet; Jenny Schecter, who has recently moved to Los Angeles to live with her boyfriend Tim Haspell; Shane McCutcheon, an androgynous, highly sexual hairstylist; Alice Pieszecki, a bisexual journalist who maintains The Chart; Dana Fairbanks, a closeted professional tennis player; and Kit Porter, Bette's straight half-sister.

Season 2
Main article: The L Word (season 2)
The second season of The L Word premiered on February 20, 2005 and ended on May 15, 2005. The season introduces Carmen de la Pica Morales, a DJ who becomes part of a love triangle with Shane and Jenny; Helena Peabody, a wealthy art patron who becomes a rival to Bette and love interest to Tina (while she and Bette are separated).

Major storylines in the season include Tina's pregnancy following a second insemination, culminating in Tina and Bette's reconciliation at the end of the season; the introduction of Mark Wayland, a documentary filmmaker who moves in with Shane and Jenny and Kit's acquisition of The Planet following Marina's departure from Los Angeles;[17] Shane and Jenny becoming the unknowing subjects of Mark's documentary after he places hidden cameras in their home; a developing relationship between Alice and Dana; and insights into Jenny's past as an abused child.

Season 3
Main article: The L Word (season 3)
The third season of The L Word premiered on January 8, 2006 and ended on March 26, 2006. The season introduces Moira Sweeney, a working-class butch; and Angus Partridge, a male nanny who becomes Kit's lover.[18]

The season is set six months after the birth of Tina and Bette's daughter Angelica. Major storylines include Bette and Tina's relationship deteriorating once again, which leads Tina to start a fake relationship with a man in order to win a possible custody battle with Bette; Moira taking the name Max after coming out as a trans man; Dana's diagnosis with and ultimate death from breast cancer;[19] and Shane and Carmen's engagement and wedding, which ends when Shane abandons Carmen at the altar. Helena is integrated into the primary group of characters as a friend rather than a rival; she acquires a movie studio, where she is entangled in a sexual harassment lawsuit that leads her mother to cut her off financially.

In the lead-up to the third season, the fan fiction website FanLib.com launched a contest where individuals could submit a piece of L Word fanfiction, with the winner's story incorporated into a scene in third-season episode.[20][21]

Season 4
Main article: The L Word (season 4)
The L Word was renewed for a fourth season on February 2, 2006,[22] and began filming on May 29, 2006.[23] The season aired from January 7, 2007 to March 25, 2007,[24] and introduces Jodi Lerner, a love interest for Bette;[25] Phyllis Kroll, Bette's closeted new boss at California Art College;[26] Paige Sobel, a love interest for Shane;[27] Tasha Williams, a former Captain in the Army National Guard and love interest for Alice; and Papi, who has slept with the most women on The Chart.[26] Karina Lombard reprises her role for two episodes.[28]

Major storylines in the season include the adaptation of Lez Girls, an article written by Jenny for The New Yorker, into a film; Bette taking a job as a dean at California Art College; and Tasha's struggle to reconcile her military service with her sexuality under don't ask, don't tell.

Season 5
Main article: The L Word (season 5)
The L Word was renewed for a fifth season on March 8, 2007, and began filming in summer 2007.[29] The season aired from January 6, 2008 to March 23, 2008 and introduces Nikki Stevens, a closeted gay actress who portrays the lead role in Lez Girls. [30] Adele Channing is also introduced, potentially by chance meeting Jenny at the Planet, and soon becoming her personal assistant. Papi and Angus were written out of the series.[31]

Major storylines in the season include Bette and Tina reconciling their relationship, Jenny being ousted from the production of Lez Girls, and Tasha's dishonorable discharge from the military.

Season 6
Main article: The L Word (season 6)
The sixth and final season of The L Word aired from January 18, 2009 to March 8, 2009.[32] The season introduces Kelly Wentworth, Bette's college roommate, who attempts to open a gallery with her; Jamie Chen, a social worker who becomes involved in a love triangle with Alice and Tasha; and Marybeth Duffy and Sean Holden, detectives with the LAPD.[33]

The season is a whodunit storyline focused on the murder of Jenny. The events of the season are depicted as a flashback leading up to the night of the crime, with each episode focused around what could have potentially motivated each character to have killed Jenny. The series concludes without revealing the identity of her murderer.

Interrogation tapes
Following the series finale of The L Word, Showtime released a series of seven short videos depicting Bette, Alice, Tina, Nikki Shane being questioned by the police over Jenny's murder. The episodes were posted weekly on Showtime's website. Showtime additionally released an interview with L Word series creator Ilene Chaiken, released in two weekly installments. In the interview, Chaiken stated that Alice went to jail for Jenny's murder, but was not necessarily guilty of the crime.[34][35]

Generation Q
Main article: The L Word: Generation Q
On July 11, 2017, it was announced a sequel series was in the works with Showtime.[36] Marja-Lewis Ryan has been selected to serve as executive producer and showrunner.[36][37][38] On January 31, 2019, Entertainment Weekly reported Showtime had picked up the sequel series for a premiere later in the year, in which Jennifer Beals, Katherine Moennig, and Leisha Hailey would reprise their roles.[39] Other sources, such as TVLine,[40] call the eight-episode order a revival, so the nature of the follow-up is unclear. The new series, titled The L Word: Generation Q, will premiere in the fall of 2019.[41]
"""

# https://en.wikipedia.org/wiki/The_L_Word

In [19]:
question_set = {
    'question': "What is Bette's job?",
    'context': context
}

In [20]:
nlp(question_set)

{'score': 0.5957592725753784, 'start': 3571, 'end': 3575, 'answer': 'dean'}

In [27]:
# context = "Kerryn was a data scientist in 2020, Kerryn was a senior consultant in 2021."

In [33]:
def nlp_qa(question, context):
    question_set = {
    'question': question,
    'context': context
    }
    print(nlp(question_set))



In [29]:
# nlp_qa(question="What was Kerryn's job in 2021?", context=context)

# Anvil

* https://anvil.works/


## Import

In [34]:
!pip install anvil-uplink

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 KB 11.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=36138a14f656efb089c1be3e416e59c02ce56c508130c63ec69949b15a3e6d32
  Stored in directory: /Users/Kerryn/Library/Caches/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size

In [56]:
import anvil.server

In [57]:
anvil.server.connect("LEZU7S6FEXEURVKXGHRG4TZU-EWYZ6Z7KA4U5AAFZ")

# LEZU7S6FEXEURVKXGHRG4TZU-EWYZ6Z7KA4U5AAFZ

## set up function

In [47]:
tlw_context = context

In [52]:
# # tells the jupyter server that this is an Anvil callable function
# @anvil.server.callable
# # define the function that does the NLP
# def answer_questions(question_text, context_text):
#     # convert to dict
#     question_set = {
#         'question': question_text,
#         'context': context_text
#     }

#     #run thro pipeline
#     results = nlp(question_set)

#     return (results['answer'])

In [59]:
# Tells the jupyter server that this is a an Anvil callable function
@anvil.server.callable
def answer_questions(question_text, context_text): 
    # Convert this to a dictionary
    question_set = {
                'question':question_text, 
                'context':context_text
               }
    # Run it through the NLP pipeline
    results = nlp(question_set)
    
    return results['answer']

In [55]:
anvil_result = answer_questions(question_text="What is Bette's sister's name?", context_text=tlw_context)

anvil_result

'Kit Porter'